# Laboratorul 7

In [1]:
import re

In [2]:
nouns_dict = {}
verbs_dict = {}
det_dict = {}
adj_dict = {}
conj_dict = {}
prep_dict = {}
pnoun_dict = {}

noun = False
verb = False
det = False
adj = False
conj = False
prep = False
pnoun = False
masculine = False
feminine = False

lexicon_file = open('lexicon.txt', 'r', encoding="utf-8")
for line in lexicon_file.readlines():
    line = line.rstrip(' \n')
    if ' N ' in line:
        noun = True
    if 'V ' in line:
        noun = False
        verb = True
    if 'DET ' in line:
        verb = False
        det = True
    if 'ADJ ' in line:
        det = False
        adj = True
    if 'CONJ ' in line:
        adj = False
        conj = True
    if 'PREP ' in line:
        conj = False
        prep = True
    if 'PNOUN ' in line:
        prep = False
        pnoun = True
    if noun:
        if 'Masc ' in line:
            masculine = True
            feminine = False
        if 'Fem ' in line:
            masculine = False
            feminine = True
        if masculine:
            line_list = line.split(' -> ')
            if 'Masc ' in line_list[0] or '' == line_list[0]:
                pass
            else:
                nouns_dict[line_list[0]] = (line_list[1], 'm')
        if feminine:
            line_list = line.split(' -> ')
            if 'Fem ' in line_list[0] or '' == line_list[0]:
                pass
            else:
                nouns_dict[line_list[0]] = (line_list[1], 'f')
    if verb:
        line_list = line.split(' -> ')
        if 'V ' in line_list[0] or '' == line_list[0]:
            pass
        else:
            verbs_dict[line_list[0]] = line_list[1]
    if det:
        line_list = line.split(' -> ')
        if 'DET ' in line_list[0] or '' == line_list[0]:
            pass
        else:
            if line_list[1] in ['Le', 'Un']:
                det_dict[(line_list[0], 'm')] = line_list[1]
            if line_list[1] in ['La', 'Une']:
                det_dict[(line_list[0], 'f')] = line_list[1]
    if adj:
        line_list = line.split(' -> ')
        if 'ADJ ' in line_list[0] or '' == line_list[0]:
            pass
        else:
            adj_dict[line_list[0]] = line_list[1]
    if conj:
        line_list = line.split(' -> ')
        if 'CONJ ' in line_list[0] or '' == line_list[0]:
            pass
        else:
            conj_dict[line_list[0]] = line_list[1]
    if prep:
        line_list = line.split(' -> ')
        if 'PREP ' in line_list[0] or '' == line_list[0]:
            pass
        else:
            prep_dict[line_list[0]] = line_list[1]
    if pnoun:
        if 'PNOUN ' in line:
            pass
        else:
            pnoun_dict[line] = line
print(nouns_dict)
print(verbs_dict)
print(det_dict)
print(adj_dict)
print(conj_dict)
print(prep_dict)
print(pnoun_dict)

{'Cat': ('Chat', 'm'), 'Book': ('Livre', 'm'), 'Sugar': ('Sucre', 'm'), 'Cane': ('Canne', 'f'), 'Woman': ('Femme', 'f'), 'Table': ('Table', 'f'), 'Saw': ('Scie', 'f')}
{'Sleeps': 'Dort', 'Reads': 'Lit', 'Walks': 'Marche', 'Cut': 'A coupé', 'Saw': 'A Vu', 'Is': 'Est'}
{('The', 'm'): 'Le', ('The', 'f'): 'La', ('A', 'm'): 'Un', ('A', 'f'): 'Une'}
{'Red': 'Rouge', 'Happy': 'Heureuse', 'Cane': 'De Canne'}
{'And': 'Et', 'Or': 'Ou'}
{'With': 'Avec', 'To': 'A', 'From': 'De', 'Under': 'Sous'}
{'Mary': 'Mary', 'Cambridge': 'Cambridge'}


In [3]:
semantic_rules = {}
rules_file = open('rules.txt', 'r', encoding="utf-8")
for line in rules_file.readlines():
    line = line.rstrip(' \n')
    line_list = line.split(' -> ')
    if len(line_list) == 1:
        pass
    else:
        pattern = '[\s+\s]+'
        key_value_list = []
        for element in line_list:
            element = element.lstrip(' ').rstrip(' ')
            tuple_words = tuple(re.split(pattern, element))
            key_value_list.append(tuple_words)
        semantic_rules[key_value_list[0]] = key_value_list[1]
semantic_rules

{('ADJ', 'N'): ('N', 'ADJ'),
 ('The', 'Masc', 'N'): ('Le', 'Masc', 'N'),
 ('The', 'Fem', 'N'): ('La', 'Fem', 'N'),
 ('A', 'Masc', 'N'): ('Un', 'Masc', 'N'),
 ('A', 'Fem', 'N'): ('Une', 'Fem', 'N'),
 ('DET', 'saw'): ('DET', 'Fem', 'N'),
 ('saw', 'DET'): ('V', 'DET'),
 ('DET', 'cane'): ('DET', 'Fem', 'N'),
 ('N', 'cane'): ('N', 'ADJ')}

In [4]:
sentences = ['Mary reads a book.', 'A book is under the table.',
             'Mary cut the sugar cane with a saw.', 'Mary cut the sugar cane and is happy.',
             'The woman with a red cane saw a cat under the table and walks to the cat.']

In [5]:
for sentence in sentences:
    translation = ''
    parts_of_speech = [] # in this list we are going to save the rules that apply to the current sentence
    words = sentence.split(' ')
    for index in range(0, len(words)):
        word = words[index]
        word = word.rstrip('.')
        word = word[0].upper() + word[1:]
        if word in nouns_dict: # check if current word is a noun
            if index >= 1:
                # compute the rules for DET + NOUN
                if nouns_dict[word][1] == 'm':
                    if words[index - 1].lower() == 'the':
                        parts_of_speech.append(('The', 'Masc', 'N'))
                    elif words[index -1].lower() == 'a':
                        parts_of_speech.append(('A', 'Masc', 'N'))
                elif nouns_dict[word][1] == 'f':
                    if words[index - 1].lower() == 'the':
                        parts_of_speech.append(('The', 'Fem', 'N'))
                    elif words[index -1].lower() == 'a':
                        parts_of_speech.append(('A', 'Fem', 'N'))
                
                if words[index - 1].lower()[0].upper() + words[index - 1].lower()[1:] in adj_dict: # check if previous word was adjective
                    parts_of_speech.append(('ADJ', 'N'))
                    
        # SPECIAL RULES FOR SAW AND CANE
        if word == 'Saw': # compute the rules for ('DET', 'saw') and ('saw', 'DET')
            gender_saw = nouns_dict[word][1]
            if (words[index - 1].lower()[0].upper() + words[index - 1].lower()[1:], gender_saw) in det_dict: # check if previous word was determinant
                parts_of_speech.append(('DET', 'saw'))
            if index < len(words) - 1:
                if (words[index + 1].lower()[0].upper() + words[index + 1].lower()[1:], 'm') in det_dict: # check if next word is determinant
                    parts_of_speech.append(('saw', 'DET'))
        if word == 'Cane': # compute the rules for ('DET', 'cane') and ('N', 'cane')
            gender_cane = nouns_dict[word][1]
            if (words[index - 1].lower()[0].upper() + words[index - 1].lower()[1:], gender_cane) in det_dict: # check if previous word was determinant
                parts_of_speech.append(('DET', 'cane'))
            if words[index - 1].lower()[0].upper() + words[index - 1].lower()[1:] in nouns_dict: # check if previous ward was a noun:
                parts_of_speech.append(('N', 'cane'))
        for word_dictionary in [nouns_dict, verbs_dict, det_dict, adj_dict, conj_dict, prep_dict, pnoun_dict]:
            if word in word_dictionary:
                if parts_of_speech != []: # we have a semantic rule
                    rule_how_to_translate = semantic_rules[parts_of_speech[-1]]
                    if rule_how_to_translate in [('Un', 'Masc', 'N'), ('Une', 'Fem', 'N'), ('Le', 'Masc', 'N'), ('La', 'Fem', 'N')]:
                        translation += rule_how_to_translate[0] + ' ' + nouns_dict[word][0] + ' '
                    # elif parts_of_speech[-1] == ('ADJ', 'N'):
                    #     translation += nouns_dict[word][0] + ' ' + adj_dict[words[index-1][0].upper() + words[index-1][1:].lower()] + ' '
                    elif rule_how_to_translate == ('V', 'DET'):
                        translation += verbs_dict[word] + ' '
                    parts_of_speech.remove(parts_of_speech[-1])  
                else:
                    # MOT-A-MOT TRANSLATION
                    if type(word_dictionary[word]) == str:
                        translated_word = word_dictionary[word]
                    translation += translated_word + ' '
                    
    translation = translation[0].upper() + translation[1:].lower().rstrip(' ') + '.'
    print(sentence)
    print(translation)
    print('\n')

Mary reads a book.
Mary lit un livre.


A book is under the table.
Un livre est sous la table.


Mary cut the sugar cane with a saw.
Mary a coupé le sucre de canne avec une scie.


Mary cut the sugar cane and is happy.
Mary a coupé le sucre de canne et est heureuse.


The woman with a red cane saw a cat under the table and walks to the cat.
La femme avec rouge de canne a vu un chat sous la table et marche a le chat.


